In [ ]:
import pandas as pd
import json 

path = "data.json"

with open(path, "r", encoding='utf-8') as f:
    conteudo = json.load(f)

# Extrair os dados da chave correta
df = pd.DataFrame(conteudo["blocks"])

print(df.head())


In [ ]:
df.columns

In [ ]:
print(df["metrics"].iloc[0])


In [ ]:
# Expandir a coluna 'metrics' em colunas separadas
metricas_expandido = pd.json_normalize(df["metrics"])

# Juntar de volta com as colunas de data
df_metricas = pd.concat([df[["block_start", "block_end", "image_count"]], metricas_expandido], axis=1)

# Renomear colunas para coincidir com o seu esquema
df_metricas = df_metricas.rename(columns={
    "block_start": "periodo_inicio",
    "block_end": "periodo_fim"
})

print(df_metricas.head())


In [ ]:
df_metricas['periodo_inicio'] = pd.to_datetime(df_metricas['periodo_inicio'])
df_metricas = df_metricas.sort_values(by='periodo_inicio')

In [ ]:
df_metricas.columns

In [ ]:
indices = [
    'BIOMASSA_PROXY_mean',
       'BIOMASSA_PROXY_median', 'BIOMASSA_PROXY_stdDev', 'EVI_mean',
       'EVI_median', 'EVI_stdDev', 'GNDVI_mean', 'GNDVI_median',
       'GNDVI_stdDev', 'NDMI_mean', 'NDMI_median', 'NDMI_stdDev', 'NDRE_mean',
       'NDRE_median', 'NDRE_stdDev', 'NDVI_mean', 'NDVI_median', 'NDVI_stdDev',
       'NDWI_mean', 'NDWI_median', 'NDWI_stdDev', 'RENDVI_mean',
       'RENDVI_median', 'RENDVI_stdDev'
]

# 4) Criar um dicionário com todas as séries temporais
series = {}

for nome in indices:
    ts = (
        df_metricas.set_index("periodo_inicio")[nome]
          .asfreq("MS")       # frequência mensal (início do mês)
          .interpolate()      # preenche eventuais falhas
    )
    series[nome] = ts

# 5) Visualizar uma amostra
for nome, ts in series.items():
    print(f"\n▶ {nome}")
    print(ts.head())

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

predicoes = {}
metricas = []  # lista para armazenar as métricas

# ---------- 1. Treinar todos os modelos e salvar métricas ----------
for nome in indices:
    y = series[nome]
    treino = y[:'2023-12-01']
    teste = y['2024-01-01':]

    modelo = SARIMAX(
        treino,
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 12),
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    resultado = modelo.fit(disp=False)

    pred = resultado.get_forecast(steps=len(teste))
    pred_mean = pred.predicted_mean

    # Calcular métricas
    mae = mean_absolute_error(teste, pred_mean)
    rmse = np.sqrt(mean_squared_error(teste, pred_mean))

    # Guardar as previsões e métricas
    predicoes[nome] = {
        "pred": pred_mean,
        "mae": mae,
        "rmse": rmse
    }

    metricas.append({
        "variavel": nome,
        "MAE": mae,
        "RMSE": rmse
    })

# ---------- 2. Criar DataFrame com todas as métricas ----------
df_metricas = pd.DataFrame(metricas)
print("📊 Métricas de todos os índices:")
print(df_metricas.sort_values("RMSE"))

# ---------- 3. Gerar os gráficos ----------
for nome in indices:
    y = series[nome]
    treino = y[:'2023-12-01']
    teste = y['2024-01-01':]
    pred_mean = predicoes[nome]["pred"]

    plt.figure(figsize=(8,4))
    plt.plot(treino, label="Treino", color="blue")
    plt.plot(teste, label="Real (Teste)", color="orange")
    plt.plot(pred_mean, label="Previsto (SARIMA)", color="red")
    plt.title(f"Previsão {nome.upper()} - SARIMA Univariado")
    plt.legend()
    plt.show()


In [ ]:
import json

# Criar lista com todas as métricas + valores previstos
metricas_com_preds = []

for nome in indices:
    pred_mean = predicoes[nome]["pred"]
    mae = predicoes[nome]["mae"]
    rmse = predicoes[nome]["rmse"]

    # Converte a série de previsões em dicionário {data: valor}
    preds_dict = {str(data.date()): float(valor) for data, valor in pred_mean.items()}

    metricas_com_preds.append({
        "variavel": nome,
        "MAE": round(mae, 4),
        "RMSE": round(rmse, 4),
        "valores_previstos": preds_dict
    })

# Caminho para salvar o JSON
caminho_json = "CAMINHO"

# Salvar o arquivo JSON formatado
with open(caminho_json, "w", encoding="utf-8") as f:
    json.dump(metricas_com_preds, f, ensure_ascii=False, indent=2)

print(f"✅ JSON salvo com sucesso em: {caminho_json}")
